# Assignment 1 : Linear Optimization

Name : Ahmik Virani <br>
Roll Number : ES22BTECH11001

In [ ]:
import pandas as pd
import numpy as np
from scipy.linalg import null_space

# Read CSV file
df = pd.read_csv('Testcase.csv', header=None)

# Get total rows and columns
rows, cols = df.shape
m = rows - 2
n = cols - 1

# Extract components
z = df.iloc[0, :-1].dropna().to_numpy(dtype=float)
c = df.iloc[1, :-1].dropna().to_numpy(dtype=float)
A = df.iloc[2:, :-1].to_numpy(dtype=float)
B = df.iloc[2:, -1].to_numpy(dtype=float)

print("n =", n)
print("m =", m)
print("z =", z)
print("c =", c)
print("A =", A)
print("B =", B)


n = 2
m = 3
z = [4.3  1.75]
c = [6. 7.]
A = [[ 2. -3.]
 [ 9.  1.]
 [-1.  1.]]
B = [ 16. 200.   0.]


In [ ]:
# This is a function which takes the initial feasable point to the vertex
def go_to_vertex(z, A, B, m, n):
  current_z = z.copy()

  while True:
    # We first need to find the set of tight and untight rows of A
    # Let A1 be the set of tight rows
    # Let A2 be the set of untight rows
    Az = A @ current_z
    A1_indexes = []
    A2_indexes = []
    for i in range(m):
        # For floats, we need to check if they are very close
        if abs(Az[i] - B[i]) < 1e-9:
            A1_indexes.append(i)
        else:
            A2_indexes.append(i)

    A1 = A[A1_indexes, :]
    A2 = A[A2_indexes, :]

    # Ensure that the number of tight rows is not zero
    if len(A1_indexes) == 0:
      rank = 0
    else:
      rank = np.linalg.matrix_rank(A1)

    # If rank of the tight rows is >= n, then this is already a vertex
    if rank >= n:
      if not np.allclose(current_z, z, atol=1e-9):
        print("Trying to reach a vertex before applying Simplex. Moving to: ", np.round(current_z, 9))
      return current_z

    # We need to find a direction in the null space of A1
    if rank == 0:
      # We need to move to a random direction
      d = np.random.rand(n)
      d = d / np.linalg.norm(d)
    else:
      d = null_space(A1)[:, 0]

    min_step_size = np.inf

    # If we go towards d or opposite d
    direction = 0

    # First check going towards d
    for i in A2_indexes:
      Ai_dot_d = np.dot(A[i], d)
      if Ai_dot_d > 1e-9:
        step_size = (B[i] - Az[i]) / Ai_dot_d
        if step_size < min_step_size:
          min_step_size = step_size
          direction = 1

    # Next check going opposite to d
    for i in A2_indexes:
      Ai_dot_d = np.dot(A[i], -d)
      if Ai_dot_d > 1e-9:
        step_size = (B[i] - Az[i]) / Ai_dot_d
        if step_size < min_step_size:
          min_step_size = step_size
          direction = -1

    current_z += min_step_size * direction * d

In [ ]:
# We are already given the starting point
current_vertex = z.copy()

# First ensure that the initial point is a vertex or not
current_vertex = go_to_vertex(current_vertex, A, B, m, n)

while(True):
  # Ensure that we are printing the answer after some rounding to handle python small errors
  print(f"Current Vertex : {np.round(current_vertex, 9)}, Objective Function = {np.round(np.dot(c, current_vertex), 9)}")

  # We first need to find the set of tight and untight rows of A
  # Let A1 be the set of tight rows
  # Let A2 be the set of untight rows
  Az = A @ current_vertex
  A1_indexes = []
  A2_indexes = []
  for i in range(m):
      # For floats, we need to check if they are very close
      if abs(Az[i] - B[i]) < 1e-9:
          A1_indexes.append(i)
      else:
          A2_indexes.append(i)

  A1 = A[A1_indexes, :]
  A2 = A[A2_indexes, :]

  # We know that rank of A1 is n
  # Also, we know that A1 has as inverse
  A_inv = np.linalg.inv(A1)

  # Next we check c.T dot A_inv
  # If even one is negative, then z is not optimum
  all_positive = True
  for row in A_inv.T:
    if np.dot(c.T, row) < -1e-9:
      all_positive = False
      # Move in opposite direction of this
      # Let d be direction of movement
      d = -row

      # We also need the size of movement
      # But we only check along the untight rows, i.e. A2
      step_sizes = []
      for i in A2_indexes:
        Ai_dot_d = np.dot(A[i], d)
        # We move in this direction if it becomes more tight
        if Ai_dot_d > 1e-9:
          step_sizes.append((B[i] - Az[i]) / Ai_dot_d)

      if step_sizes:
          # We choose the minimum step size to ensure we are going to the adjacent vertex
          alpha = min(step_sizes)
          current_vertex += alpha * d
          break

  if(all_positive):
    break

Trying to reach a vertex before applying Simplex. Moving to:  [-16. -16.]
Current Vertex : [-16. -16.], Objective Function = -208.0
Current Vertex : [20. 20.], Objective Function = 260.0
